In [52]:
import os
import sys
import pandas as pd
import numpy as np
import nltk
from IPython.display import display

# grab the csvs
pmb = pd.read_csv('fact_pmb.csv')
opp = pd.read_csv('operasional_public_pendaftar.csv')
schools = pd.read_csv('daftar_sekolah.csv')
# start making the pd
dataset = pmb.filter(items=['kode_pendaftar'])

## Highschool Background Filtering (DW FTI)

kolum id_lokasi + kabupaten/kota + provinsi<br>
not java: 1 - 153 & 258 - 491 <br>
java : 154 - 257

In [53]:
# test grab location id
dataset['id_lokasi'] = pmb.filter(items=['id_lokasi'])
# make a boolean array to indicate if the number is as the criteria
dataset['lokasi'] = np.where(dataset.id_lokasi.between(154,257),1,0)
print(dataset)

      kode_pendaftar  id_lokasi  lokasi
0             150002        201       1
1             150013        213       1
2             150015        215       1
3             150017        215       1
4             150018        213       1
...              ...        ...     ...
2639          210546        215       1
2640          210566         81       0
2641          210577        206       1
2642          210766        430       0
2643          210928        461       0

[2644 rows x 3 columns]


In [54]:
#test status sekolah (negeri/swasta)
dataset['tipe_sekolah_asal'] = pmb.filter(items=['tipe_sekolah_asal'])
dataset['status'] = np.where(dataset.tipe_sekolah_asal == 1,1,0)
print(dataset)

      kode_pendaftar  id_lokasi  lokasi  tipe_sekolah_asal  status
0             150002        201       1                  2       0
1             150013        213       1                  2       0
2             150015        215       1                  2       0
3             150017        215       1                  2       0
4             150018        213       1                  2       0
...              ...        ...     ...                ...     ...
2639          210546        215       1                  1       1
2640          210566         81       0                  1       1
2641          210577        206       1                  1       1
2642          210766        430       0                  1       1
2643          210928        461       0                  1       1

[2644 rows x 5 columns]


In [55]:
# test tipe sekolah (SMA/SMK)
dataset['jurusan_sekolah'] = pmb.filter(items=['jurusan_sekolah'])
dataset['tipe'] = np.where(dataset.jurusan_sekolah == "LAINNYA",0,1)
print(dataset)

      kode_pendaftar  id_lokasi  lokasi  tipe_sekolah_asal  status  \
0             150002        201       1                  2       0   
1             150013        213       1                  2       0   
2             150015        215       1                  2       0   
3             150017        215       1                  2       0   
4             150018        213       1                  2       0   
...              ...        ...     ...                ...     ...   
2639          210546        215       1                  1       1   
2640          210566         81       0                  1       1   
2641          210577        206       1                  1       1   
2642          210766        430       0                  1       1   
2643          210928        461       0                  1       1   

     jurusan_sekolah  tipe  
0                IPA     1  
1                IPA     1  
2                IPA     1  
3                IPA     1  
4             

In [56]:
dataset.drop(['id_lokasi','tipe_sekolah_asal','jurusan_sekolah'], inplace=True, axis=1)
print(dataset)

      kode_pendaftar  lokasi  status  tipe
0             150002       1       0     1
1             150013       1       0     1
2             150015       1       0     1
3             150017       1       0     1
4             150018       1       0     1
...              ...     ...     ...   ...
2639          210546       1       1     1
2640          210566       0       1     1
2641          210577       1       1     1
2642          210766       0       1     1
2643          210928       0       1     1

[2644 rows x 4 columns]


## Adding Id of highschools
objective: 
1. add "sekolah_asal" from opp to pmb
2. add "id_daftar_sekolah" from schools to pmb

In [57]:
# grab the school names
id_reg = dataset.filter(items=['kode_pendaftar'])
print(id_reg)
school_names = opp.filter(items=["kode_pendaftar", "sekolah_asal"])
school_id = schools.filter(items=["id_daftar_sekolah","sekolah_asal"])
print(school_names)
print(school_id)

      kode_pendaftar
0             150002
1             150013
2             150015
3             150017
4             150018
...              ...
2639          210546
2640          210566
2641          210577
2642          210766
2643          210928

[2644 rows x 1 columns]
       kode_pendaftar                      sekolah_asal
0              150001          SMA NEGERI 1 MUARA BUNGO
1              150002              SMA SEDES SAPIENTIAE
2              150003       SMA NEGERI 1 BENGKULU UTARA
3              150004              SMA SEDES SAPIENTIAE
4              150005      SMA STELLA DUCE 2 YOGYAKARTA
...               ...                               ...
10551          211494  SMA FRATER DON BOSCO BANJARMASIN
10552          211495         SMA KATOLIK SANTO ANDREAS
10553          211505               SMK TELKOM MAKASSAR
10554          211523              SMA NEGERI 1 MERAUKE
10555          211524               SMA NEGERI 1 MIMIKA

[10556 rows x 2 columns]
      id_daftar_sekolah  

In [58]:
output = pd.merge(id_reg, school_names, on="kode_pendaftar", how="left") 
output2 = pd.merge(school_id, output, on="sekolah_asal", how="right")
print(output)
print(output2)

      kode_pendaftar                  sekolah_asal
0             150002          SMA SEDES SAPIENTIAE
1             150013  SMA STELLA DUCE 2 YOGYAKARTA
2             150015       SMA BOPKRI 1 YOGYAKARTA
3             150017          SMA KOLESE DE BRITTO
4             150018          SMA KOLESE DE BRITTO
...              ...                           ...
2639          210546       SMA NEGERI 2 YOGYAKARTA
2640          210566       SMA NEGERI 14 PEKANBARU
2641          210577          SMA NEGERI 1 PARAKAN
2642          210766           SMA NEGERI 1 MAMASA
2643          210928           SMA NEGERI 1 WAMENA

[2644 rows x 2 columns]
      id_daftar_sekolah                  sekolah_asal  kode_pendaftar
0                2181.0          SMA SEDES SAPIENTIAE          150002
1                   3.0  SMA STELLA DUCE 2 YOGYAKARTA          150013
2                   7.0       SMA BOPKRI 1 YOGYAKARTA          150015
3                   8.0          SMA KOLESE DE BRITTO          150017
4            

In [59]:
dataset['id_daftar_sekolah'] = output2['id_daftar_sekolah']
dataset['id_prodi_pilihan_1'] = pmb['id_prodi_pilihan_1']
dataset['id_prodi_pilihan_2'] = pmb['id_prodi_pilihan_2']
dataset['nilai_tpa_verbal'] = pmb['nilai_tpa_verbal']
dataset['nilai_tpa_spasial'] = pmb['nilai_tpa_spasial']
dataset['nilai_tpa_analogi'] = pmb['nilai_tpa_analogi']
dataset['nilai_tpa_numerik'] = pmb['nilai_tpa_numerik']
dataset['avg_nilai_uan'] = pmb['avg_nilai_uan'] 
dataset['avg_nilai_raport'] = pmb['avg_nilai_raport']
dataset['is_lolos_seleksi'] = pmb['is_lolos_seleksi']
dataset['id_prodi_diterima'] = pmb['id_prodi_diterima']
dataset['id_daftar_sekolah']=dataset['id_daftar_sekolah'].fillna(0)
dataset['id_daftar_sekolah']=dataset['id_daftar_sekolah'].astype(int)
print(dataset)

      kode_pendaftar  lokasi  status  tipe  id_daftar_sekolah  \
0             150002       1       0     1               2181   
1             150013       1       0     1                  3   
2             150015       1       0     1                  7   
3             150017       1       0     1                  8   
4             150018       1       0     1                  8   
...              ...     ...     ...   ...                ...   
2639          210546       1       1     1                424   
2640          210566       0       1     1               1628   
2641          210577       1       1     1                996   
2642          210766       0       1     1               2176   
2643          210928       0       1     1                472   

      id_prodi_pilihan_1  id_prodi_pilihan_2  nilai_tpa_verbal  \
0                      9                   1                 0   
1                      9                   1                 0   
2                    

In [60]:
dataset = dataset.drop('kode_pendaftar',axis=1)

## Cleaning the target variable

In [61]:
dataset['is_diterima'] = np.where((dataset.id_prodi_diterima == 9) | (dataset.id_prodi_diterima == 10),1,0)
dataset

,lokasi,status,tipe,id_daftar_sekolah,id_prodi_pilihan_1,id_prodi_pilihan_2,nilai_tpa_verbal,nilai_tpa_spasial,nilai_tpa_analogi,nilai_tpa_numerik,avg_nilai_uan,avg_nilai_raport,is_lolos_seleksi,id_prodi_diterima,is_diterima
0,1,0,1,2181,9,1,0,0,0,0,NaN,NaN,1,9,1
1,1,0,1,3,9,1,0,0,0,0,NaN,NaN,1,9,1
2,1,0,1,7,9,1,0,0,0,0,NaN,NaN,1,9,1
3,1,0,1,8,9,1,0,0,0,0,NaN,NaN,1,9,1
4,1,0,1,8,9,10,0,0,0,0,NaN,NaN,1,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2639,1,1,1,424,9,1,0,0,0,0,0.0,0.0,1,9,1
2640,0,1,1,1628,9,1,0,0,0,0,0.0,0.0,1,9,1
2641,1,1,1,996,9,3,0,0,0,0,0.0,0.0,1,9,1
2642,0,1,1,2176,9,1,0,0,0,0,0.0,0.0,1,9,1


In [62]:
dataset = dataset.drop(['is_lolos_seleksi','id_prodi_diterima'],axis=1)
dataset.to_csv('dataset.csv', encoding='utf-8', index=False)

njp = dataset[dataset['nilai_tpa_verbal'] > 0]
jp = dataset[dataset['nilai_tpa_verbal'] == 0]

## Cleaning and stuff

In [63]:
njp = njp.drop(columns = ['avg_nilai_uan', 'avg_nilai_raport'])
jp = jp.drop(columns = ['nilai_tpa_verbal','nilai_tpa_spasial','nilai_tpa_numerik','nilai_tpa_analogi'])

In [64]:
jp.to_csv('data_jp.csv', encoding='utf-8', index=False)
njp.to_csv('data_njp.csv', encoding='utf-8', index=False)

In [65]:
jp['avg_nilai_raport'].fillna(0, inplace=True)
jp['avg_nilai_uan'].fillna(0, inplace=True)
# terlalu banyak data rata2 nilai raportnya kosong
jp_legit = jp[(jp['avg_nilai_raport'] > 0) | (jp['is_diterima'] == 0)]

In [66]:
jp_legit.loc[jp_legit.avg_nilai_uan > 100 , 'avg_nilai_uan'] = jp_legit[jp_legit['avg_nilai_uan'] > 100]['avg_nilai_uan']/4

In [67]:
jp_legit['avg_nilai_raport'] = jp_legit['avg_nilai_raport'].astype(int)
jp_legit.to_csv('data_jp_legit.csv', encoding='utf-8', index=False)

C:\Users\joshu\AppData\Local\Temp\ipykernel_6076\1897843106.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jp_legit['avg_nilai_raport'] = jp_legit['avg_nilai_raport'].astype(int)


## Normalization for Regular Registration Data

In [68]:
njp.iloc[:,6:10]

,nilai_tpa_verbal,nilai_tpa_spasial,nilai_tpa_analogi,nilai_tpa_numerik
43,100,90,70,90
48,10,10,10,10
53,150,90,120,80
60,100,120,120,80
61,10,10,10,20
...,...,...,...,...
2063,100,20,60,100
2064,20,50,90,70
2066,70,100,20,20
2067,10,60,20,20


In [17]:
from scipy.stats import zscore
tpa_zscore = zscore(njp.iloc[:,6:10])
tpa_zscore

,nilai_tpa_verbal,nilai_tpa_spasial,nilai_tpa_analogi,nilai_tpa_numerik
43,0.756498,0.322359,0.087438,0.434869
48,-0.979664,-1.368159,-1.171673,-1.072579
53,1.721032,0.322359,1.136698,0.246438
60,0.756498,0.956303,1.136698,0.246438
61,-0.979664,-1.368159,-1.171673,-0.884148
...,...,...,...,...
2063,0.756498,-1.156845,-0.122414,0.623300
2064,-0.786758,-0.522900,0.507142,0.058007
2066,0.177777,0.533673,-0.961821,-0.884148
2067,-0.979664,-0.311586,-0.961821,-0.884148


In [26]:
njp.iloc[:,6:10] = tpa_zscore

In [27]:
njp.to_csv('data_njp.csv', encoding='utf-8', index=False)